## 2. Tensorflow Low Level API

### 2.3. 变量Variables

1. 与张量对象不同，变量[tf.Variable]()对象是可变的，即对于单次执行`tf.Session.run`，变量的值是可变的，[tf.Variable]()存在于单次 `tf.session.run`调用的上下文之外。
2. 在Tensorflow内部，[tf.Variable]()存储持久性张量。特定操作(op) 可以读取和修改此张量的值。这些修改在多个 [tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)之间是可见的，因此对于一个[tf.Variable]()，多个工作器worker(分布式模型的概念)可以看到相同的值。

[tf.Variable]: https://www.tensorflow.org/api_docs/python/tf/Variable;

**创建变量Creating a Variable**

---

**创建变量的最佳方式是调用[tf.get_variable](https://www.tensorflow.org/api_docs/python/tf/get_variable)函数。**此函数要求您指定变量的名称。其他副本将使用此名称访问同一变量，以及在对模型设置检查点和导出模型时指定此变量的值。[tf.get_variable]()还允许您重复使用先前创建的同名变量，从而轻松定义重复利用层的模型。

```
tensorflow.get_variable = get_variable(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=None, collections=None, caching_device=None, partitioner=None, v
alidate_shape=True, use_resource=None, custom_getter=None, constraint=None, synchronization=<VariableSynchronization.AUTO: 0>, aggregation=<VariableAggregation.NONE: 0>)
    Gets an existing variable with these parameters or create a new one.

    This function prefixes the name with the current variable scope
    and performs reuse checks. See the
    [Variable Scope How To](https://tensorflow.org/guide/variables)
    for an extensive description of how reusing works. Here is a basic example:

    ``python
    def foo():
      with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("v", [1])
      return v

    v1 = foo()  # Creates v.
    v2 = foo()  # Gets the same, existing v.
    assert v1 == v2
    ``

    If initializer is `None` (the default), the default initializer passed in
    the variable scope will be used. If that one is `None` too, a
    `glorot_uniform_initializer` will be used. The initializer can also be
    a Tensor, in which case the variable is initialized to this value and shape.

    Similarly, if the regularizer is `None` (the default), the default regularizer
    passed in the variable scope will be used (if that is `None` too,
    then by default no regularization is performed).

    If a partitioner is provided, a `PartitionedVariable` is returned.
    Accessing this object as a `Tensor` returns the shards concatenated along
    the partition axis.

    Some useful partitioners are available.  See, e.g.,
    `variable_axis_size_partitioner` and `min_max_variable_partitioner`.

    Args:
      name: The name of the new or existing variable.
      shape: Shape of the new or existing variable.
      dtype: Type of the new or existing variable (defaults to `DT_FLOAT`).
      initializer: Initializer for the variable if one is created. Can either be an initializer object or a Tensor. If it's a Tensor, its shape must be known
        unless validate_shape is False.
      regularizer: A (Tensor -> Tensor or None) function; the result of
        applying it on a newly created variable will be added to the collection
        `tf.GraphKeys.REGULARIZATION_LOSSES` and can be used for regularization.
      trainable: If `True` also add the variable to the graph collection
        `GraphKeys.TRAINABLE_VARIABLES` (see `tf.Variable`).
      collections: List of graph collections keys to add the Variable to.
        Defaults to `[GraphKeys.GLOBAL_VARIABLES]` (see `tf.Variable`).
      caching_device: Optional device string or function describing where the
        Variable should be cached for reading.  Defaults to the Variable's
        device.  If not `None`, caches on another device.  Typical use is to
        cache on the device where the Ops using the Variable reside, to
        deduplicate copying through `Switch` and other conditional statements.
      partitioner: Optional callable that accepts a fully defined `TensorShape`
        and `dtype` of the Variable to be created, and returns a list of
        partitions for each axis (currently only one axis can be partitioned).
      validate_shape: If False, allows the variable to be initialized with a
          value of unknown shape. If True, the default, the shape of initial_value
          must be known. For this to be used the initializer must be a Tensor and
          not an initializer object.
      use_resource: If False, creates a regular Variable. If true, creates an
        experimental ResourceVariable instead with well-defined semantics.
        Defaults to False (will later change to True). When eager execution is
        enabled this argument is always forced to be True.
      custom_getter: Callable that takes as a first argument the true getter, and
        allows overwriting the internal get_variable method.
        The signature of `custom_getter` should match that of this method,
        but the most future-proof version will allow for changes:
        `def custom_getter(getter, *args, **kwargs)`.  Direct access to
        all `get_variable` parameters is also allowed:
        `def custom_getter(getter, name, *args, **kwargs)`.  A simple identity
        custom getter that simply creates variables with modified names is:
        ``python
        def custom_getter(getter, name, *args, **kwargs):
          return getter(name + '_suffix', *args, **kwargs)
        ``
      constraint: An optional projection function to be applied to the variable
        after being updated by an `Optimizer` (e.g. used to implement norm
        constraints or value constraints for layer weights). The function must
        take as input the unprojected Tensor representing the value of the
        variable and return the Tensor for the projected value
        (which must have the same shape). Constraints are not safe to use when doing asynchronous distributed training.
      synchronization: Indicates when a distributed a variable will be
        aggregated. Accepted values are constants defined in the class
        `tf.VariableSynchronization`. By default the synchronization is set to
        `AUTO` and the current `DistributionStrategy` chooses
        when to synchronize. If `synchronization` is set to `ON_READ`,
        `trainable` must not be set to `True`.
      aggregation: Indicates how a distributed variable will be aggregated.
        Accepted values are constants defined in the class
        `tf.VariableAggregation`.

    Returns:
      The created or existing `Variable` (or `PartitionedVariable`, if a
      partitioner was used).

    Raises:
      ValueError: when creating a new variable and shape is not declared,
        when violating reuse during variable creation, or when `initializer` dtype
        and `dtype` don't match. Reuse is set inside `variable_scope`.
```

使用参数`name`和`shape`来创建变量：

In [1]:
import tensorflow as tf

my_variable = tf.get_variable("my_variable", [1, 2, 3])

print(my_variable, type(my_variable))

<tf.Variable 'my_variable:0' shape=(1, 2, 3) dtype=float32_ref> <class 'tensorflow.python.ops.variables.RefVariable'>


这将创建一个名为“my_variable”的变量，该变量是形状为`[1, 2, 3]`的三维张量。默认情况下，此变量将具有dtype [tf.float32](https://www.tensorflow.org/api_docs/python/tf#float32)，其初始值将通过[tf.glorot_uniform_initializer](https://www.tensorflow.org/api_docs/python/tf/glorot_uniform_initializer)随机设置。

使用参数`dtype`和`initializer`来创建变量：

参数`initializer`可以是
1. `initializer`初始化器对象；
2. `shape`已知的`tf.Tensor`张量对象，无需再传`shape`参数；

In [2]:
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32, initializer=tf.zeros_initializer)
print(my_int_variable, type(my_int_variable))

<tf.Variable 'my_int_variable:0' shape=(1, 2, 3) dtype=int32_ref> <class 'tensorflow.python.ops.variables.RefVariable'>


TensorFlow 提供了许多方便的初始化器。或者，您也可以将tf.Variable初始化为tf.Tensor的值。例如：

In [3]:
other_variable = tf.get_variable("other_variable", dtype=tf.int32, initializer=tf.constant([23, 43]))

**变量集合Variable collections**

由于TensorFlow程序的未连接部分之间可能需要创建变量，因此能有一种方式访问所有变量很重要。为此，TensorFlow提供了**集合**，

**集合是张量或其他对象（如[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)实例）的命名列表。**

默认情况下，每个[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)都放置在以下两个集合中：

+ [tf.GraphKeys.GLOBAL_VARIABLES](https://www.tensorflow.org/api_docs/python/tf/GraphKeys#GLOBAL_VARIABLES) - 多台设备间共享的变量，
+ [tf.GraphKeys.TRAINABLE_VARIABLES](https://www.tensorflow.org/api_docs/python/tf/GraphKeys#TRAINABLE_VARIABLES) - 可训练的变量，TensorFlow将计算其梯度的变量。

如果您不希望变量可训练，可以将其添加到[tf.GraphKeys.LOCAL_VARIABLES](https://www.tensorflow.org/api_docs/python/tf/GraphKeys#LOCAL_VARIABLES)集合中。



```
 |  All variables are automatically collected in the graph where they are
 |  created. By default, the constructor adds the new variable to the graph
 |  collection `GraphKeys.GLOBAL_VARIABLES`. The convenience function
 |  `global_variables()` returns the contents of that collection.
 |
 |  When building a machine learning model it is often convenient to distinguish
 |  between variables holding the trainable model parameters and other variables
 |  such as a `global step` variable used to count training steps. To make this
 |  easier, the variable constructor supports a `trainable=<bool>` parameter. If
 |  `True`, the new variable is also added to the graph collection
 |  `GraphKeys.TRAINABLE_VARIABLES`. The convenience function
 |  `trainable_variables()` returns the contents of this collection. The
 |  various `Optimizer` classes use this collection as the default list of
 |  variables to optimize.
```


In [4]:
# 添加变量至`tf.GraphKeys.LOCAL_VARIABLES`集合的方式一
my_local = tf.get_variable("my_local", shape=(), collections=[tf.GraphKeys.LOCAL_VARIABLES])
# 添加变量至`tf.GraphKeys.LOCAL_VARIABLES`集合的方式二
my_non_trainable = tf.get_variable("my_non_trainable", shape=(), trainable=False)

也可以将变量添加至自己的集合。集合名称可为任何字符串，且无需显式创建集合。

[tf.add_to_collection](https://www.tensorflow.org/api_docs/python/tf/add_to_collection)可以将变量添加至某一集合：

In [5]:
tf.add_to_collection("my_collection_name", my_local)

[tf.get_collection]()返回某个集合中所有变量的列表：

In [6]:
tf.get_collection("my_collection_name")

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]

**设备放置方式(分布式模型的内容)**

与任何TensorFlow操作一样，您可以将变量放置在特定设备上。

In [7]:
# 以下代码段创建了名为 v 的变量并将其放置在第二个 GPU 设备上：
with tf.device("device:GPU:1"):
    v = tf.get_variable("v", [1])

在分布式模型设置中，将变量放置在正确设备上尤为重要。如果不小心将变量放在工作器worker而不是参数服务器parameter server上，可能会严重减慢训练速度，最坏的情况下，可能会让每个工作器worker不断复制各个变量。为此，我们提供了[tf.train.replica_device_setter](https://www.tensorflow.org/api_docs/python/tf/train/replica_device_setter)，它可以自动将变量放置在参数服务器parameter server中。例如：

In [3]:

cluster_spec = {
    "ps": ["ps0:2222", "ps1:2222"],
    "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]
}

with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
    v = tf.get_variable("v", shape=[20, 20])  # this variable is placed in the parameter server by the replica_device_setter

**初始化变量Initializing variables**

---

1. 在Tensorflow Low Level API中，变量创建后，必须先初始化才可以使用；
2. [tf.contrib.slim](https://www.tensorflow.org/api_docs/python/tf/contrib/slim)、[tf.estimator.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator)和`Keras`等大多数高级框架在训练模型前会自动初始化变量。

显式初始化在其他方面很有用。它允许
1. 从检查点重新加载模型时不用重新运行潜在资源消耗大的初始化器
2. 在分布式设置中共享随机初始化的变量时具有确定性。allowing determinism when randomly-initialized variables are shared in a distributed setting.

变量初始化方法：
1. 使用变量的初始化器属性：初始化某一变量；
2. 使用assign操作为变量赋值；
3. 使用`tf.global_variables_initializer`：初始化[tf.GraphKeys.GLOBAL_VARIABLES](https://www.tensorflow.org/api_docs/python/tf/GraphKeys#GLOBAL_VARIABLES)集合中所有变量;

```
 |  When you launch the graph, variables have to be explicitly initialized before
 |  you can run Ops that use their value. You can initialize a variable by
 |  running its *initializer op*, restoring the variable from a save file, or
 |  simply running an `assign` Op that assigns a value to the variable. In fact,
 |  the variable *initializer op* is just an `assign` Op that assigns the
 |  variable's initial value to the variable itself.
 |
 |  ``python
 |  # Launch the graph in a session.
 |  with tf.Session() as sess:
 |      # Run the variable initializer.
 |      sess.run(w.initializer)
 |      # ...you now can run ops that use the value of 'w'...
 |  ``
 |
 |  The most common initialization pattern is to use the convenience function
 |  `global_variables_initializer()` to add an Op to the graph that initializes
 |  all the variables. You then run that Op after launching the graph.
 |
 |  ``python
 |  # Add an Op to initialize global variables.
 |  init_op = tf.global_variables_initializer()
 |
 |  # Launch the graph in a session.
 |  with tf.Session() as sess:
 |      # Run the Op that initializes global variables.
 |      sess.run(init_op)
 |      # ...you can now run any Op that uses variable values...
 |  ``
 |
 |  If you need to create a variable with an initial value dependent on another
 |  variable, use the other variable's `initialized_value()`. This ensures that
 |  variables are initialized in the right order.
```

In [ ]:
sess = tf.Session()
# 初始化某一变量
sess.run(my_varialbe.initializer)
# 初始化global所有变量
sees.run(tf.global_variables_initializer())

`tf.report_uninitialized_variables()`:查询未初始化变量

In [ ]:
print(sess.run(tf.report_uninitilized_variables()))

默认情况下，[tf.global_variables_initializer](https://www.tensorflow.org/api_docs/python/tf/initializers/global_variables)不会指定变量的初始化顺序。因此，如果变量的初始值取决于另一变量的值，那么很有可能会出现错误。任何时候，如果您在并非所有变量都已初始化的上下文中使用某个变量值（例如在初始化某个变量时使用另一变量的值），最好使用 `variable.initialized_value()`，而非`variable`

**使用变量Using Variables**

---

如张量对象[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)一样使用变量对象[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)，使用方法`assign`和`assign_add`或者[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)类中的友元来为变量赋值；

In [1]:
import tensorflow as tf
# 像张量一样使用变量
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = v + 1  #  w is a tf.Tensor which is computed based on the value of v.
           # Any time a variable is used in an expression it gets automatically
           # converted to a tf.Tensor representing its value.
        
# 为变量赋值
k = tf.get_variable("k", shape=(), initializer=tf.zeros_initializer())
with tf.Session() as sess:
    assignment = k.assign_add(1)
    tf.global_variables_initializer().run()
    sess.run(assignment)  # or assignment.op.run(), or assignment.eval()

大多数TensorFlow优化器都有专门的操作(op)，这些操作会根据某种梯度下降算法有效地更新变量的值。请参阅[tf.train.Optimizer](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer)，了解如何使用优化器。

由于变量是可变的，因此及时了解任意时间点所使用的变量值版本有时十分有用。要在事件发生后强制重新读取变量的值，可以使用 [tf.Variable.read_value](https://www.tensorflow.org/api_docs/python/tf/Variable#read_value)。例如：

In [1]:
import tensorflow as tf
j = tf.get_variable("j", shape=(), initializer=tf.zeros_initializer())
assignment = j.assign_add(1)
with tf.control_dependencies([assignment]):
    w = j.read_value()  # w is guaranteed to reflect j's value after the assign_add operation
    print(w, type(w))

Tensor("read:0", shape=(), dtype=float32) <class 'tensorflow.python.framework.ops.Tensor'>


**共享变量Sharing variables**

---

两种共享变量的方式：

+ 显式传递[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)对象;
+ 编写Tensorflow函数，将[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)对象隐式封装在[tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope)变量作用域对象内。
    1. 使用[tf.variable_scope]()的参数`reuse=True`和相同作用域名称;
    2. 使用[tf.variable_scope]()的实例方法`scope.reuse_variables()`;

变量作用域允许您在调用隐式创建和使用变量的函数时控制变量重用。作用域还允许您以分层和可理解的方式命名变量。

例如，假设我们编写一个函数来创建一个卷积/relu 层：

In [2]:
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights"
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    # Create variable named "biases"
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.))
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

此函数使用短名称`weights`和`biases`，这有利于清晰区分二者。然而，在真实模型中，我们需要很多此类卷积层，而且重复调用此函数将不起作用：

In [3]:
input1 = tf.random_normal([1, 10, 10, 32])
input2 = tf.random_normal([1, 20, 20, 32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
# x = conv_relu(x, kernel_shape=[5, 5, 32, 32], bias_shape=[32])  # This fails.

由于期望的操作不清楚（创建新变量还是重新使用现有变量？），因此 TensorFlow 将会失败。不过，在不同作用域内调用`conv_relu`可表明我们想要创建新变量：

In [4]:
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

使用开头介绍的方法，共享变量：

In [5]:
with tf.variable_scope("model"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model", reuse=True):
    output2 = my_image_filter(input2)

In [8]:
with tf.variable_scope("model2") as scope:
    output1 = my_image_filter(input1)
    scope.reuse_variables()
    output2 = my_image_filter(input2)

由于依赖于作用域的确切字符串名称可能比较危险，因此也可以根据另一作用域初始化某个变量作用域：

In [9]:
with tf.variable_scope("model3") as scope:
    output1 = my_image_filter(input1)
with tf.variable_scope(scope, reuse=True):
    output2 = my_image_filter(input2)